# Combining range and direction for improved localizatioin

This notebook can be used to reproduced the results shown in the paper of the above title, as well as to visualize new simulated data.

In [ ]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, asin, acos, atan
%reload_ext autoreload
%autoreload 2
from matplotlib import rc
rc('font',**{'family':'dejavu-sans','sans-serif':['Helvetica']})
rc('text', usetex=True)

# read results
def parse_results(methods, times, n_its):
    linestyles = ['-','-.',':','--','-']
    markerstyle = ('o', 's', '^', '*', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
    dict_methods = {m:{'rmses':'','linestyle':linestyles[i], 'marker':markerstyle[i]} 
                    for i,m in enumerate(methods)}
    name = 'rmses_{}_{}'
    for key in dict_methods.keys():
        total_n_it = 0
        for time in times:
            try: 
                new_data = np.load(FOLDER+'/'+name.format(key, time)+'.npy')
                print('loading {}/'.format(FOLDER) +name.format(key, time)+'.npy')
                try:
                    dict_methods[key]['rmses'] += n_its[time] * new_data
                except TypeError:
                    dict_methods[key]['rmses'] = n_its[time] * new_data
                total_n_it += n_its[time]
            except:
                print('could not find new data under', FOLDER+'/'+name.format(key, time)+'.npy')
                raise
        if total_n_it > 0:
            dict_methods[key]['rmses'] /= total_n_it
        else:
            raise NameError('Could not find any results for method {}'.format(key))
    return dict_methods

# find times with correct options:
def get_times(**kwargs):
    times = []
    n_its = {}
    good_options = None
    for f in os.listdir(FOLDER):
        if f.startswith('options_'):
            filename = '{}/{}'.format(FOLDER, f)
            with open(filename, 'r') as options_file:
                options = json.load(options_file)
            options_test = [options[key]==val for key, val in kwargs.items()]
            if all(options_test):
                time = f[-15:-5]
                n_its[str(time)] = options['n_it']
                times.append(time)
                good_options = options
    print('identified times:',times)
    print('n_its',n_its)
    return times, n_its, good_options

## Recreation of plots used in the paper

### Read results

In [ ]:
import os
import json
from run_simulation import parse_options, parse_options_gaussian

FOLDER = 'icassp_results' # original version
N = 6
d = 2

# Choose which methods to load. 
# Set to parameters used in run_simulation, when creating
# the results you would like to consider.
methods = ['CDM','constrained E-MDS','E-MDS','MDS']

# get results with correct parameters
times, n_its, options = get_times(N=N, n_it=500, d=d)
dict_methods = parse_results(methods, times, n_its)
sigmas, rhos, __ = parse_options_gaussian(options)

### Plot results against distance error

In [ ]:
from plots import plot_against_distance

chosen_rho = [2] #[2,10] #[0, 2, 10]
saveas = '{}/low_angle_noise_N{}_d{}_deg.eps'.format(FOLDER, N, d)
title = '"low" angle noise: $\sigma_\\alpha={0:.2f}$ (${1:.1f}^\circ$)'
plot_against_distance(dict_methods, chosen_rho, rhos, sigmas, saveas, title, legend=True)
f = plt.gcf(); f.set_size_inches(10,5)

chosen_rho = [10] #[2,10] #[0, 2, 10]
saveas = '{}/high_angle_noise_N{}_d{}_deg.eps'.format(FOLDER, N, d)
title = '"high" angle noise: $\sigma_\\alpha={0:.2f}$ (${1:.1f}^\circ)$'
plot_against_distance(dict_methods, chosen_rho, rhos, sigmas, saveas, title)
f = plt.gcf(); f.set_size_inches(10,5)

### Plot results against angle error

In [ ]:
from plots import plot_against_angles

chosen_sig = [1] #[0, 9] #[0, 3, 9]
title = '"low" distance noise: $\sigma_d={:2.2f}$'
saveas = '{}/low_distance_noise_N{}_d{}_deg2.eps'.format(FOLDER, N, d)
plot_against_angles(dict_methods, chosen_sig, sigmas, rhos, saveas, title, legend=True, gaussian=options['gaussian'])
f = plt.gcf(); f.set_size_inches(10,5)

chosen_sig = [3] #[0, 9] #[0, 3, 9]
title = '"high" distance noise: $\sigma_d={:2.2f}$'
saveas = '{}/high_distance_noise_N{}_d{}_deg2.eps'.format(FOLDER, N, d)
plot_against_angles(dict_methods, chosen_sig, sigmas, rhos, saveas, title, gaussian=options['gaussian'])
f = plt.gcf(); f.set_size_inches(10,5)

In [ ]:
from plots import plot_seaborn

plot_seaborn(dict_methods, options, 'CDM', folder=FOLDER, ylabel=True, vmin=0, vmax=.4, cbar=False, square=True)
plot_seaborn(dict_methods, options, 'constrained E-MDS', folder=FOLDER, vmin=0, vmax=.4, square=True)

difference = dict_methods['CDM']['rmses'] - dict_methods['constrained E-MDS']['rmses']
plot_seaborn(dict_methods, options, 'difference CDM - constrained E-MDS', folder=FOLDER, matrix=difference, figsize=(10,5), ylabel=True, center=0, square=False)

## Create own results

To create your own results, run the script run_simulation, choosing the desired parameters. Then, you can load and plot the results here (or wherever you wish) as shown above. 

If you wish to implement your own algorithms and compare, we recommend that you create a fork of the pylocus package (that way it can stay private at first if you wish) and add the method to the algorithms.py script, borrowing from the other examples for the correct syntax. Using a local install of your forked package, you can easily compare with the provided methods. 

In [ ]:
#FOLDER = 'many_iterations' # new version with M*V
FOLDER = 'many_iterations_original' # original with M*K

methods = ['CDM','constrained E-MDS','E-MDS','MDS']

# get results with correct parameters
times, n_its, good_options = get_times(N=6, n_it=100, d=2)
dict_methods = parse_results(methods, times, n_its)

sigmas, rhos, __ = parse_options_gaussian(good_options)
chosen_rho = [2] #[2,10] #[0, 2, 10]
saveas = '{}/low_angle_noise_N{}_d{}_deg.eps'.format(FOLDER, N, d)
title = '"low" angle noise: $\sigma_\\alpha={0:.2f}$ (${1:.1f}^\circ$)'
plot_against_distance(dict_methods, chosen_rho, rhos, sigmas, saveas, title, legend=True)

difference = dict_methods['CDM']['rmses'] - dict_methods['constrained E-MDS']['rmses']
plot_seaborn(dict_methods, options, 'difference CDM - constrained E-MDS', folder=FOLDER, matrix=difference, figsize=(10,5), ylabel=True, center=0, square=False)